In [6]:
import pandas as pd
import numpy as np
import re
from math import sqrt, log
from typing import Union

In [7]:
class PySRModelLoader:
    def __init__(self, model_path: str):
        self.model_path = model_path
        self.equations_df = pd.read_csv(model_path)
        self.models = self._build_model_functions()

    def _build_model_functions(self):
        model_funcs = {}
        for _, row in self.equations_df.iterrows():
            complexity = row['Complexity']
            eq_str = row['Equation']
            eq_str_fixed = eq_str.replace("^", "**")
            required_vars = self._extract_variable_names(eq_str_fixed)
            func = self._create_function(eq_str_fixed, required_vars)
            model_funcs[complexity] = {
                'func': func,
                'equation': eq_str_fixed,
                'required_vars': required_vars,
                'loss': row['Loss']
            }
        return model_funcs

    def _extract_variable_names(self, equation: str):
        tokens = re.findall(r'\b[a-zA-Z_][a-zA-Z0-9_]*\b', equation)
        known_funcs = {'sqrt', 'log'}
        variables = sorted(set(t for t in tokens if t not in known_funcs and not t.isnumeric()))
        return variables

    def _create_function(self, equation_str, required_vars):
        def safe_sqrt(x):
            try:
                return sqrt(x) if x >= 0 else 0
            except:
                return 0

        def safe_log(x):
            try:
                return log(x) if x > 0 else 0
            except:
                return 0

        def model_func(row):
            try:
                local_vars = {var: row.get(var, np.nan) for var in required_vars}
                return eval(
                    equation_str,
                    {"__builtins__": None, "sqrt": safe_sqrt, "log": safe_log},
                    local_vars
                )
            except Exception as e:
                print(f"[!] Eval error with expression: {equation_str} — {e}")
                return np.nan
        return model_func
    
    def get_best_model(self):
        return min(self.models.items(), key=lambda x: x[1]['loss'])[1]

    def predict(self, features_df: "pd.DataFrame", complexity: int = None) -> "pd.Series":
        """
        Predicts for a whole DataFrame of features using specified model complexity.
        If none is passed, use the best one (lowest loss).
        """
        if complexity is None:
            model = self.get_best_model()
        else:
            model = self.models.get(complexity)
            if model is None:
                raise ValueError(f"No model found with complexity: {complexity}")

        func = model["func"]
        return features_df.apply(lambda row: func(row.to_dict()), axis=1)

In [8]:
def predict(self, features_df: pd.DataFrame, complexity: int = None):
    if complexity is None:
        model = self.get_best_model()
    else:
        model = self.models.get(complexity)
        if model is None:
            raise ValueError(f"No model found with complexity: {complexity}")

    func = model['func']
    # This MUST return a Series
    return features_df.apply(lambda row: pd.to_numeric(func(row.to_dict()), errors='coerce'), axis=1)


In [9]:
class FeatureDatasetLoader:
    def __init__(self, feature_csv_path: str):
        self.df = pd.read_csv(feature_csv_path)

        # Remove infs
        self.df = self.df.replace([np.inf, -np.inf], np.nan)

        # Drop only if key prediction features are missing
        key_features = ['rolling_pts_avg_5G', 'rolling_fga_5G', 'adjusted_pts_proj_def', 'pts_to_3pm_ratio', 'shots_per_min']
        self.df = self.df.dropna(subset=key_features)

    def get_features(self):
        return self.df


In [10]:
if __name__ == "__main__":
    # Load feature data
    feature_loader = FeatureDatasetLoader("nba_player_features_rolling5G.csv")
    features_df = feature_loader.get_features()
    #print(features_df.columns.tolist())
    print(f"Loaded {len(features_df)} rows.")

    if features_df.empty:
        print("[!] Feature DataFrame is empty. Cannot proceed with prediction.")
    else:
        # Load PySR equations
        model_loader = PySRModelLoader("hall_of_fame.csv")

        # 🔍 SANITY CHECK ON FIRST ROW
        best_model = model_loader.get_best_model()
        test_row = features_df.iloc[0].to_dict()

        #print("\n[Best Model]")
        #print("Equation:", best_model["equation"])
        #print("Required Vars:", best_model["required_vars"])
        #print("Row Sample:")
        #print({k: test_row.get(k, 'Missing') for k in best_model["required_vars"]})

        try:
            pred = best_model["func"](test_row)
            #print("[✓] First prediction result:", pred)
        except Exception as e:
            print("[!] Error evaluating first prediction:", e)

        # Predict with best model
        predictions = model_loader.predict(features_df)
        if isinstance(predictions, pd.DataFrame):
            print("[!] Warning: Predict function returned a DataFrame instead of Series")
            predictions = predictions.iloc[:, 0]  # Use first column if so

        features_df["pysr_prediction"] = predictions

        print(features_df[['PLAYER_NAME', 'GAME_DATE', 'rolling_pts_avg_5G', 'pysr_prediction', 'PTS']].head(20))
        features_df.to_csv("predictions_with_pysr.csv", index=False)
        print("[✓] Predictions saved to predictions_with_pysr.csv")

Loaded 756 rows.
     PLAYER_NAME   GAME_DATE  rolling_pts_avg_5G  pysr_prediction  PTS
0   Jayson Tatum  2024-11-01                30.2        25.783174   32
1   Jayson Tatum  2024-11-02                29.2        25.867267   29
2   Jayson Tatum  2024-11-04                30.0        25.986841   28
3   Jayson Tatum  2024-11-06                28.2        24.807545   32
4   Jayson Tatum  2024-11-08                31.6        26.266011   33
5   Jayson Tatum  2024-11-10                30.8        25.920747   31
6   Jayson Tatum  2024-11-12                30.6        25.905277   20
7   Jayson Tatum  2024-11-13                28.8        25.069355   36
8   Jayson Tatum  2024-11-16                30.4        25.729967   24
9   Jayson Tatum  2024-11-19                28.8        24.935457   33
10  Jayson Tatum  2024-11-22                28.8        24.638637   16
11  Jayson Tatum  2024-11-24                25.8        23.490942   26
12  Jayson Tatum  2024-11-25                27.0        24.0